In [44]:
!pip install tensorflow-recommenders
!pip install tensorflow
!pip install pandas
!pip install scikit-learn


In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load the dataset (adjust the path based on your environment)
df = pd.read_csv('exercise_dataset.csv')

# Display the first few rows of the dataset
print(df.head())

# Check for missing values
print(df.isnull().sum())


   id  age  height  weight  fitness_goal workout_level  \
0   0   23     169      87  build muscle      advanced   
1   1   50     182      87   lose weight      beginner   
2   2   33     177      62   lose weight  intermediate   
3   3   25     172      75  build muscle      beginner   
4   4   29     163      51      maintain      beginner   

                       health_conditions  \
0                          Heart Disease   
1                            Sleep Apnea   
2  Gastroesophageal Reflux Disease(GERD)   
3                                 Anemia   
4                            Cholesterol   

                          recommend_workout  
0                   crunches, lunges, plank  
1  lunges, plank, pushups, crunches, squats  
2  lunges, plank, pushups, crunches, squats  
3            squats, plank, lunges, pushups  
4                   crunches, lunges, plank  
id                      0
age                     0
height                  0
weight                  0
fitnes

In [46]:
# Fill any missing values in the 'health_conditions' column with 'unknown'
df['health_conditions'] = df['health_conditions'].fillna('unknown')

# Drop unnecessary columns if any (e.g., 'id')
df = df.drop(columns=['id'], errors='ignore')  # Assuming 'id' column exists, modify if needed


In [47]:
# Check for missing values
print(df.isnull().sum())

age                  0
height               0
weight               0
fitness_goal         0
workout_level        0
health_conditions    0
recommend_workout    0
dtype: int64


In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf



# Define features and target variable
X = df[['age', 'height', 'weight']]
y = df['recommend_workout']

# One-hot encode categorical variables
encoder = OneHotEncoder(sparse_output=False)
encoded_categorical = encoder.fit_transform(df[['fitness_goal', 'workout_level', 'health_conditions']])

# Combine features
X = np.hstack((X.values, encoded_categorical))

# Encode target variable (recommend_workout) into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert y to categorical
num_classes = len(np.unique(y_encoded))
y_categorical = tf.keras.utils.to_categorical(y_encoded, num_classes)




In [49]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Adjust num_classes for your output
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


Epoch 1/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7616 - loss: 0.7926 - val_accuracy: 0.9735 - val_loss: 0.0888
Epoch 2/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9663 - loss: 0.0955 - val_accuracy: 0.9701 - val_loss: 0.0746
Epoch 3/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9734 - loss: 0.0717 - val_accuracy: 0.9784 - val_loss: 0.0482
Epoch 4/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9795 - loss: 0.0525 - val_accuracy: 0.9800 - val_loss: 0.0453
Epoch 5/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9790 - loss: 0.0500 - val_accuracy: 0.9797 - val_loss: 0.0541
Epoch 6/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9806 - loss: 0.0480 - val_accuracy: 0.9823 - val_loss: 0.0397
Epoch 7/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9833 - loss: 0.0383 - val_accuracy: 0.9874 - val_loss: 0.0307
Epoch 8/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9844 - loss: 0.0355 - 

In [51]:

# Example predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
predicted_workouts = label_encoder.inverse_transform(predicted_classes)

print("Predicted workouts:", predicted_workouts)

438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Predicted workouts: ['lunges, plank, pushups, crunches, squats' 'crunches, lunges, plank'
 'squats, plank, lunges, pushups' ...
 'lunges, plank, pushups, crunches, squats'
 'lunges, plank, pushups, crunches, squats'
 'lunges, plank, pushups, crunches, squats']


In [52]:
# Save the model
model.save('workout_recommendation_model.h5')


In [56]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('workout_recommendation_model.h5')

# Sample data for recommendations
sample_data = pd.DataFrame({
    'age': [26],
    'height': [170],
    'weight': [72],
    'fitness_goal': ['build muscle'],
    'workout_level': ['beginner'],
    'health_conditions': ['none']
})

# Preprocess the sample data
# One-hot encode the categorical variables
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # Handle unknown categories

# Note: You need to fit the encoder on the original training data's categorical variables
# If you've saved the encoder during training, load it; else, you can refit it
# Here, we'll fit it again for demonstration purposes
encoder.fit(data[['fitness_goal', 'workout_level', 'health_conditions']])

encoded_sample_categorical = encoder.transform(sample_data[['fitness_goal', 'workout_level', 'health_conditions']])

# Combine numerical features with encoded categorical features
sample_X = np.hstack((sample_data[['age', 'height', 'weight']].values, encoded_sample_categorical))

# Make predictions
predictions = model.predict(sample_X)

# Decode the predicted classes
predicted_classes = np.argmax(predictions, axis=1)

# Load or fit a label encoder to decode workout recommendations
label_encoder = LabelEncoder()
label_encoder.fit(data['recommend_workout'])  # Re-fit on training data if needed
predicted_workouts = label_encoder.inverse_transform(predicted_classes)

# Output the recommendations
sample_data['recommended_workout'] = predicted_workouts
print("\nRecommended Workouts:")
print(sample_data[['age', 'height', 'weight', 'fitness_goal', 'workout_level', 'health_conditions', 'recommended_workout']])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step

Recommended Workouts:
   age  height  weight  fitness_goal workout_level health_conditions  \
0   26     170      72  build muscle      beginner              none   

              recommended_workout  
0  squats, plank, lunges, pushups  
